Hier werden die Portfolio-Methoden angewandt und mithilfe der Performance-Metriken bewertet. Die Ergebnisse werden in CSV-Dateien festgehalten. Das Akquirieren, modifizieren und beschreiben der Daten befindet sich der Übersicht halber im Notebook Datenbasis. Die Daten werden dort als CSV exportiert und hier als Dataframe wieder eingelesen.

Die Korrektheit der implementierten Methoden und Metriken wird in dem separaten Notebook Verifizierung anhand eines kleinen Teilbestandes aufgezeigt, siehe dazu auch beigefügte Excel-Dateien, in denen die Portfolio-Gewichte "per Hand" nachgerechnet und mit den Ergebnissen aus der Python-Implementierung abgeglichen werden.

ACHTUNG: LANGE LAUFZEIT!

## Importe

In [1]:
#Import der notwendigen Bibliotheken, Installation mit pip bereits erfolgt

import yfinance as yf
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy.optimize import minimize
import math
import random
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import linkage, dendrogram
import riskfolio as rp
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import squareform
from riskfolio import HCPortfolio as hc
from pandas.tseries.offsets import MonthBegin
from pandas.tseries.offsets import MonthEnd

## Einlesen der Daten

In [2]:
#Bearbeitete JSE-Daten (Gesamtbestand)

df_JSE = pd.read_csv(
    'df_JSE_Final_V7.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [3]:
df_JSE.info

<bound method DataFrame.info of             4SI.JO   ABG.JO  ACL.JO  ACT.JO       ADH.JO  AEL.JO  \
Date                                                               
2005-06-01     NaN   8240.0  4980.0   130.0   133.701508     NaN   
2005-06-02     NaN   8250.0  4949.0   130.0   131.749695     NaN   
2005-06-03     NaN   8261.0  4949.0   130.0   131.749695     NaN   
2005-06-06     NaN   8290.0  5023.0   130.0   132.725601     NaN   
2005-06-07     NaN   8345.0  4845.0   149.0   131.749695     NaN   
...            ...      ...     ...     ...          ...     ...   
2025-05-26    70.0  16845.0   109.0   160.0  3199.000000  2350.0   
2025-05-27    70.0  16848.0   105.0   160.0  3210.000000  2350.0   
2025-05-28    73.0  17300.0   101.0   174.0  3283.000000  2355.0   
2025-05-29    70.0  17579.0   106.0   166.0  3247.000000  2362.0   
2025-05-30    70.0  17332.0   116.0   160.0  3215.000000  2358.0   

                  AFE.JO  AFH.JO  AFT.JO        AGL.JO  ...  VKE.JO   VOD.JO  \
Dat

In [4]:
#Bearbeitete SP500-Daten (Gesamtbestand)

df_SP500 = pd.read_csv(
    'df_SP500_Final_V7.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    parse_dates=['Date'],
    dayfirst=True
)

In [5]:
df_SP500.info

<bound method DataFrame.info of                      A  AAL  AAP        AAPL        ABBV        ABNB  \
Date                                                                   
2005-06-01   17.296137  NaN  NaN    1.439286         NaN         NaN   
2005-06-02   17.303289  NaN  NaN    1.430000         NaN         NaN   
2005-06-03   17.210300  NaN  NaN    1.365714         NaN         NaN   
2005-06-06   17.539343  NaN  NaN    1.354286         NaN         NaN   
2005-06-07   17.324751  NaN  NaN    1.305000         NaN         NaN   
...                ...  ...  ...         ...         ...         ...   
2025-05-23  108.529999  NaN  NaN  195.270004  183.259995  126.720001   
2025-05-27  111.260002  NaN  NaN  200.210007  185.720001  129.399994   
2025-05-28  110.879997  NaN  NaN  200.419998  183.089996  128.669998   
2025-05-29  113.279999  NaN  NaN  199.949997  185.619995  128.360001   
2025-05-30  111.919998  NaN  NaN  200.850006  186.110001  129.000000   

                   ABT       AC

In [6]:
#Reduzierter SP500-Bestand ohne ausgeschiedene Unternehmen und auf Größe des JSE runtergebrochen

df_SP500_reduziert = pd.read_csv(
    'sp500_gefiltert_neuneu.csv',
    sep=',',
    encoding='utf-8',
    index_col=0,
    decimal = '.',
    low_memory=False,
    dayfirst=True
)

In [7]:
df_SP500_reduziert.info

<bound method DataFrame.info of                      A        AAPL         ABT         ACN         ADI  \
Date                                                                     
2005-06-01   17.296137    1.439286         NaN         NaN         NaN   
2005-06-02   17.303289    1.430000         NaN         NaN         NaN   
2005-06-03   17.210300    1.365714         NaN         NaN         NaN   
2005-06-06   17.539343    1.354286         NaN         NaN         NaN   
2005-06-07   17.324751    1.305000         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2025-05-23  108.529999  195.270004  131.300003  309.579987  210.470001   
2025-05-27  111.260002  200.210007  132.940002  315.429993  216.889999   
2025-05-28  110.879997  200.419998  132.020004  315.989990  215.619995   
2025-05-29  113.279999  199.949997  132.850006  317.730011  215.750000   
2025-05-30  111.919998  200.850006  133.580002  316.820007  213.979996   

     

## Definition der Methoden und Metriken

#### Equally-Weighted Portfolio

In [8]:
#Rückgabe gleicher Gewichte anhand der Anzahl der Unternehmen (Input Returns)

def equally_weighted(returns):
    anzahl_assets = returns.shape[1]

    w0 = 1/anzahl_assets
    gewichte = np.full(anzahl_assets, w0)

    return gewichte

#### Minimum-Variance Portfolio

In [9]:
#Minimum-Variance Portfolio
#Code adaptiert von https://medium.com/@BorisGerat/mean-variance-and-minimum-variance-portfolio-models-in-python-64a5c6b57b2d

def min_variance_opt(matrix):
    
    #Compute the minimum-variance portfolio weights subject to the constraints of:
    #- No short-selling (weights must be >= 0)
    #- Full investment (sum of weights must equal 1)
    
    number_of_assets = matrix.shape[0]

    # Define the objective function to minimize portfolio variance
    def objective(w):
        portfolio_variance = np.dot(w, matrix @ w)
        return portfolio_variance

    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(number_of_assets)]

    # Initial guess: Equal weights for all assets
    w0 = np.ones(number_of_assets) / number_of_assets

    # Solve the optimization problem using Sequential Least Squares Quadratic Programming (SLSQP)
    res = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return res.x  # Return the optimal weights

#### Maximum Diversification Portfolio

In [10]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def calc_max_diversification_ratio(weights, matrix):
    
    # gewichtete Volatilität
    average_vol = np.dot(np.sqrt(np.diag(matrix)), weights.T)
    
    # portfolio volatilität
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_vol = np.sqrt(portfolio_variance)
    
    #Bestimmung Ratio
    diversification_ratio = average_vol/portfolio_vol
    
    # Negativer Wert für Minimierung (Maximizieren = Minimieren -)
    return -diversification_ratio

In [11]:
#Code von https://thequantmba.wordpress.com/2017/06/06/max-diversification-in-python/ adaptiert

def max_diversification_portfolio(matrix):
    
    anzahl_assets = matrix.shape[0]
    
    w0 = 1/anzahl_assets
    w = np.full(anzahl_assets, w0)
    
    # Constraint: The sum of portfolio weights must be equal to 1
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    
    # Bounds: Each weight must be between 0 and 1 (no short-selling)
    bounds = [[0, 1] for _ in range(anzahl_assets)]
    
    res = minimize(calc_max_diversification_ratio, w, bounds=bounds, args=matrix, method='SLSQP', constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    return res.x

#### Inverse Volatility Portfolio

In [12]:
#Code adaptiert von https://www.kaggle.com/code/vijipai/lesson-4-traditional-portfolio-construction-method

def inverse_volatility_portfolio (matrix):
    
    standardabweichungen = np.sqrt(np.diagonal(matrix))
    invertierte_std = 1/standardabweichungen
    summe_invertierte_std = np.sum(invertierte_std)
    
    inverse_volatility_gewichte = invertierte_std / summe_invertierte_std
    
    return inverse_volatility_gewichte

#### Equal Risk Contribution Portfolio

In [13]:
#Code adaptiert von https://medium.com/@deepml1818/building-a-python-based-risk-parity-portfolio-for-trading-40441ecdd84d

def calculate_portfolio_volatility(weights, matrix):
    
    portfolio_variance = np.dot(weights, matrix @ weights)
    portfolio_volatility = np.sqrt(portfolio_variance)
    return portfolio_volatility

def calculate_risk_contribution(weights, matrix):
    portfolio_volatility = calculate_portfolio_volatility(weights, matrix)
    marginal_contribution_zaehler = np.dot(matrix, weights)
    risk_contribution = np.multiply(weights, marginal_contribution_zaehler) / portfolio_volatility
    return risk_contribution

def risk_parity_objective(weights, matrix):
    risk_contribution = calculate_risk_contribution(weights, matrix)
    target_risk = np.mean(risk_contribution)
    return np.sum((risk_contribution - target_risk) ** 2)

In [14]:
def get_risk_parity_weights(matrix):
    
    number_of_assets = matrix.shape[0]
    w0 = 1/number_of_assets
    initial_weights = np.full(number_of_assets, w0)
    
    constraints = ({'type': 'eq', 'fun': lambda initial_weights: np.sum(initial_weights) - 1})
    bounds = [[0, 1] for _ in range(number_of_assets)]
    
    result = minimize(risk_parity_objective, initial_weights, args=(matrix), method='SLSQP', bounds=bounds, constraints=constraints, options={'ftol': 1e-12, 'disp': True, 'maxiter': 1000})
    
    return result.x

#### Hierarchical Risk Parity

In [15]:
#Aus dem Paper von Lopez De Prado (2016)
def getIVP(cov,**kargs): # Compute the inverse-variance portfolio 
    ivp=1./np.diag(cov) 
    ivp/=ivp.sum() 
    return ivp

In [16]:
#------------------------------------------------------------------------------ 
def getClusterVar(cov,cItems): # Compute variance per cluster 
    cov_=cov.loc[cItems,cItems] # matrix slice 
    w_=getIVP(cov_).reshape(-1,1) 
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0] 
    return cVar 

In [17]:
#------------------------------------------------------------------------------ 
def getQuasiDiag(link): # Sort clustered items by distance 
    link=link.astype(int) 
    sortIx=pd.Series([link[-1,0],link[-1,1]]) 
    numItems=link[-1,3] # number of original items 
    while sortIx.max()>=numItems: 
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space 
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems 
        sortIx[i]=link[j,0] # item 1 
        df0=pd.Series(link[j,1],index=i+1) 
        sortIx = pd.concat([sortIx, df0]) # item 2  ####Anpassung wegen neuer Pandas-Version!
        sortIx=sortIx.sort_index() # re-sort 
        sortIx.index=range(sortIx.shape[0]) # re-index 
    return sortIx.tolist() 

In [18]:
#------------------------------------------------------------------------------ 
def getRecBipart(cov,sortIx): # Compute HRP allocation
    w = pd.Series(1,index=sortIx) 
    cItems=[sortIx] # initialize all items in one cluster 
    while len(cItems)>0: 
        #Bi-Section der Cluster
        cItems = [i[j:k] 
                  for i in cItems 
                  for j, k in ((0, len(i)//2), (len(i)//2, len(i))) 
                  if len(i) > 1]
        for i in range(0, len(cItems), 2): # parse in pairs 
            cItems0=cItems[i] # cluster 1 
            cItems1=cItems[i+1] # cluster 2 
            cVar0=getClusterVar(cov,cItems0) 
            cVar1=getClusterVar(cov,cItems1) 
            alpha=1-cVar0/(cVar0+cVar1) 
            w[cItems0]*=alpha # weight 1 
            w[cItems1]*=1-alpha # weight 2 
    return w 

In [19]:
#------------------------------------------------------------------------------ 
def correlDist(corr): 
    # A distance matrix based on correlation, where 0<=d[i,j]<=1 
    # This is a proper distance metric 
    dist=((1-corr)/2.)**.5 # distance matrix 
    
    #Adjustierung zur condensed_distance_matrix
    condensed_distance_matrix = squareform(X=dist, checks=False)
    
    return condensed_distance_matrix

In [20]:
def hrp_main(corr, cov, returns): 
    
    #3) cluster 
    dist=correlDist(corr) 
    link=sch.linkage(dist,'single') 
    sortIx=getQuasiDiag(link) 
    sortIx=corr.index[sortIx].tolist() # recover labels 
    df0=corr.loc[sortIx,sortIx] # reorder 
    
    #4) Capital allocation 
    hrp=getRecBipart(cov,sortIx) 
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    hrp_sortiert = hrp.reindex(original_reihenfolge)
    
    array = hrp_sortiert.T.values

    return array

#### Hierarchical Equal Risk Contribution Portfolio

In [21]:
#Es wird auf die Implementierung der Riskfolio-Lib zurückgegriffen
#Code adaptiert von #Code adaptiert von https://medium.com/@orenji.eirl/hierarchical-equal-risk-contribution-with-python-and-riskfolio-lib-ec45dd0f9899

def herc (returns):

    # Building the portfolio object
    port = hc(returns=returns)

    # Estimate optimal portfolio:

    model='HERC'
    correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          correlation=correlation,
                          rm=rm,
                          rf=rf,
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

In [22]:
def herc_ledoit (returns):

    #Building the portfolio object
    port = hc(returns=returns)
    
    shrinkage_herc = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix_herc = pd.DataFrame(
        shrinkage_herc.covariance_,
        index = returns.columns,
        columns = returns.columns)

    #Estimate optimal portfolio:

    model='HERC'
    #correlation = 'pearson'
    rm = 'MV'
    rf = 0 
    linkage = 'ward' 
    max_k = 10 
    leaf_order = True 

    gewichte = port.optimization(model=model,
                          custom_cov = shrinkage_matrix_herc,
                          codependence='custom_cov',
                          rm=rm,
                          rf=rf,
                          method_cov='custom_cov',   #gemäß https://riskfolio-lib.readthedocs.io/en/latest/hcportfolio.html
                          linkage=linkage,
                          max_k=max_k,
                          leaf_order=leaf_order)
    
    #Sortierung der Gewichte zurück zur Reihenfolge der Returns
    original_reihenfolge = returns.columns
    gewichte_sortiert = gewichte.reindex(original_reihenfolge)
    
    array = gewichte_sortiert.values.flatten()

    return array

#### Vergleichs-Metriken

In [23]:
def gesamt_return(gewichte, returns):

    daily_returns = returns @ gewichte
    total = np.prod(1 + daily_returns) - 1
    
    return total

In [24]:
def portfolio_return (gewichte, returns):
    
    avg_returns = []
    avg_returns = returns.mean()
    
    pf_return = sum([gewicht * rendite for gewicht, rendite in zip(gewichte, avg_returns)])
    
    return pf_return

In [25]:
def portfolio_variance (gewichte, returns):
    daily_returns = []
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    pf_var = np.var(daily_returns)
    
    return pf_var

In [26]:
def portfolio_volatility (gewichte, returns):
    variance = portfolio_variance(gewichte, returns)
    volatility = np.sqrt(variance)
    
    return volatility

In [27]:
def annualized_return(gewichte, returns):
    daily_returns = returns @ gewichte
    annual = (np.prod(1 + daily_returns))**(252/daily_returns.shape[0]) - 1
    
    return annual

In [28]:
def annualized_portfolio_return(gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    
    return pf_return*252

In [29]:
def annualized_portfolio_volatility(gewichte, returns):
    pf_volatility = portfolio_volatility(gewichte, returns)
    
    return pf_volatility*np.sqrt(252)

In [30]:
def sharpe_ratio (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [31]:
def sharpe_ratio_one (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [32]:
def sharpe_ratio_two (gewichte, returns):
    pf_return = portfolio_return (gewichte, returns)
    volatility = portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [33]:
def annualized_sharpe_ratio (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [34]:
def annualized_sharpe_ratio_one (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.01
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [35]:
def annualized_sharpe_ratio_two (gewichte, returns):
    pf_return = annualized_portfolio_return (gewichte, returns)
    volatility = annualized_portfolio_volatility (gewichte, returns)
    zins_risikofrei = 0.02
    
    sharpe_ratio = (pf_return - zins_risikofrei) / volatility
    
    return sharpe_ratio

In [36]:
def sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [37]:
def sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [38]:
def sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [39]:
def annualized_sortino_ratio (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [40]:
def annualized_sortino_ratio_one (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.01
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [41]:
def annualized_sortino_ratio_two (gewichte, returns):    
    daily_returns = []
    zins_risikofrei = 0.02
    
    for index, zeile in returns.iterrows():
        daily_returns.append(np.dot(gewichte, zeile))
    
    daily_returns = np.array(daily_returns)
        
    temp = np.minimum(0, daily_returns - zins_risikofrei)**2
    temp = np.mean(temp)
    
    downside_dev = np.sqrt(temp)
    
    pf_return = annualized_portfolio_return (gewichte, returns)
    
    sortino_ratio = (pf_return - zins_risikofrei) / downside_dev
    
    return sortino_ratio

In [42]:
def maximum_drawdown (gewichte, returns):
    daily_returns = returns @ gewichte
    returns_kumuliert = (1 + daily_returns).cumprod()
    hochpunkte = returns_kumuliert.cummax()
    
    drawdowns = (returns_kumuliert - hochpunkte) / hochpunkte
    max_drawdown = drawdowns.min()
    
    #print(drawdowns)
    return max_drawdown

In [43]:
def calmar_ratio (gewichte, returns):
    pf_return = portfolio_return(gewichte, returns)
    max_drawdown = maximum_drawdown(gewichte, returns)
    
    calmar_ratio = pf_return / (-max_drawdown)
    
    return calmar_ratio

In [44]:
def value_at_risk_func(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = np.percentile(sorted_daily_returns, 100*0.05)
    return value_at_risk

In [45]:
#https://github.com/malctaylor15/VaR-and-Expected-Shortfall-Python/blob/master/Expected%20Shortfall%20and%20Value%20at%20Risk.ipynb

def expected_shortfall(gewichte, returns):
    daily_returns = returns @ gewichte
    sorted_daily_returns = daily_returns.sort_values(ascending = True)
    value_at_risk = value_at_risk_func(gewichte, returns)
    
    expected_shortfall = daily_returns[daily_returns <= value_at_risk].mean()
    
    return expected_shortfall

#### Sammel-Funktionen zum Berechnen und Speichern der Metriken

In [46]:
def metriken_equally_weighted(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile1 = {"Methode": "Equally-Weighted",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile1[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile1])], ignore_index=True)

In [47]:
def metriken_minimum_variance(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile2 = {"Methode": "Minimum_Variance",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile2[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile2])], ignore_index=True)

In [48]:
def metriken_max_diversification(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile3 = {"Methode": "Max-Diversification",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile3[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile3])], ignore_index=True)

In [49]:
def metriken_inverse_volatility(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile4 = {"Methode": "Inverse-Volatility",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile4[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile4])], ignore_index=True)

In [50]:
def metriken_risk_parity(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile5 = {"Methode": "Risk-Parity",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile5[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile5])], ignore_index=True)

In [51]:
def metriken_hrp(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile6 = {"Methode": "HRP",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile6[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile6])], ignore_index=True)

In [52]:
def metriken_herc(gewichte, returns, start_datum, end_datum, df):
    
    ergebnisse = np.zeros(10)
    
    metriken = ["Return","Mean_Return","Variance","Volatility","Sharpe","Sortino","Max Draw","Calmar","VaR","ES"]
    
    ergebnisse[0] = gesamt_return(gewichte, returns)
    ergebnisse[1] = portfolio_return (gewichte, returns)
    ergebnisse[2] = portfolio_variance (gewichte, returns)
    ergebnisse[3] = portfolio_volatility(gewichte, returns)
    ergebnisse[4] = sharpe_ratio (gewichte, returns)
    ergebnisse[5] = sortino_ratio (gewichte, returns)
    ergebnisse[6] = maximum_drawdown (gewichte, returns)
    ergebnisse[7] = calmar_ratio (gewichte, returns)
    ergebnisse[8] = value_at_risk_func(gewichte, returns)
    ergebnisse[9] = expected_shortfall(gewichte, returns)
    
    neue_zeile7 = {"Methode": "HERC",
                "Startdatum": start_datum,
               "Enddatum": end_datum}
    
    for m, e in zip(metriken, ergebnisse):
        neue_zeile7[m] = e
    
    # Anhängen
    return pd.concat([df, pd.DataFrame([neue_zeile7])], ignore_index=True)

## Erzeugung Ergebnisse S&P500

In [53]:
#Tägliche Renditen für den gesamten Zeitraum von 20 Jahren SP500

returns = df_SP500.pct_change()
returns = returns.where(~df_SP500.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A  AAL  AAP      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2005-06-02  0.000414  NaN  NaN -0.006452       NaN       NaN -0.001448   
2005-06-03 -0.005374  NaN  NaN -0.044955       NaN       NaN  0.000207   
2005-06-06  0.019119  NaN  NaN -0.008368       NaN       NaN  0.003729   
2005-06-07 -0.012235  NaN  NaN -0.036393       NaN       NaN  0.006398   
2005-06-08  0.009496  NaN  NaN  0.010399       NaN       NaN -0.002051   
...              ...  ...  ...       ...       ...       ...       ...   
2025-05-23 -0.010485  NaN  NaN -0.030244  0.003944 -0.007286 -0.001521   
2025-05-27  0.025154  NaN  NaN  0.025298  0.013424  0.021149  0.012490   
2025-05-28 -0.003415  NaN  NaN  0.001049 -0.014161 -0.005641 -0.006920   
2025-05-29  0.021645  NaN  NaN -0.002345  0.013818 -0.002409  0.006287   
2025-05-30 -0.012006  NaN  NaN  0.004501  0.002640  0.004986  0.005495   

     

In [54]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 36
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 189
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    gewichte_equally_weighted = equally_weighted(training_daten)
    gewichte_min_variance= min_variance_opt(cov_matrix)
    gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": gewichte_equally_weighted,
    "Minimum-Variance": gewichte_min_variance,
    "Maximum-Diversification": gewichte_max_diversification,
    "Inverse-Volatility": gewichte_inverse_volatility,
    "Equal-Risk-Contribution": gewichte_risk_parity,
    "HRP": gewichte_hrp,
    "HERC": gewichte_herc
}
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
# Alle Testperioden zusammenführen
portfolio_returns_df = pd.concat(portfolio_returns_list)
portfolio_returns_df = portfolio_returns_df.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.0443087635876786e-05
            Iterations: 263
            Function evaluations: 68380
            Gradient evaluations: 263
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5983971191200643
            Iterations: 48
            Function evaluations: 12530
            Gradient evaluations: 48
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.93551873223873e-09
            Iterations: 30
            Function evaluations: 7801
            Gradient evaluations: 30
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.22540750824403e-05
            Iterations: 266
            Function evaluations: 68894
            Gradient evaluations: 266
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.555

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9592319056214231
            Iterations: 27
            Function evaluations: 7180
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.225288083048592e-10
            Iterations: 79
            Function evaluations: 20936
            Gradient evaluations: 79
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00011019302592398058
            Iterations: 165
            Function evaluations: 43890
            Gradient evaluations: 165
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9573934670801787
            Iterations: 30
            Function evaluations: 8007
            Gradient evaluations: 30
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.3995834037116006e-10
      

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.1349627729577064e-10
            Iterations: 78
            Function evaluations: 21451
            Gradient evaluations: 78
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00011805643317614701
            Iterations: 149
            Function evaluations: 40976
            Gradient evaluations: 149
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9065979696645454
            Iterations: 29
            Function evaluations: 8000
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.104445920021461e-10
            Iterations: 78
            Function evaluations: 21451
            Gradient evaluations: 78
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.000

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0001037240438379576
            Iterations: 135
            Function evaluations: 38610
            Gradient evaluations: 135
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8243605255325261
            Iterations: 31
            Function evaluations: 8887
            Gradient evaluations: 31
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.140350032497073e-10
            Iterations: 78
            Function evaluations: 22309
            Gradient evaluations: 78
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0001031268967881135
            Iterations: 164
            Function evaluations: 47232
            Gradient evaluations: 164
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.81

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9920594940629888
            Iterations: 31
            Function evaluations: 9607
            Gradient evaluations: 31
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.818933820827119e-10
            Iterations: 57
            Function evaluations: 17614
            Gradient evaluations: 57
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.8626124052035225e-05
            Iterations: 196
            Function evaluations: 60956
            Gradient evaluations: 196
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.992149552633056
            Iterations: 33
            Function evaluations: 10292
            Gradient evaluations: 33
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2629765784387696e-09
      

...

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.802453951943493e-05
            Iterations: 198
            Function evaluations: 77221
            Gradient evaluations: 198
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.1141848036285587
            Iterations: 27
            Function evaluations: 10556
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.67257569929774e-10
            Iterations: 49
            Function evaluations: 19111
            Gradient evaluations: 49
2017-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.99223928019202e-05
            Iterations: 199
            Function evaluations: 77809
            Gradient evaluations: 199
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.104

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.194512021716391
            Iterations: 27
            Function evaluations: 10583
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1820795102419642e-09
            Iterations: 59
            Function evaluations: 23069
            Gradient evaluations: 59
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.333462909913932e-05
            Iterations: 230
            Function evaluations: 90390
            Gradient evaluations: 230
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.1947795756228112
            Iterations: 26
            Function evaluations: 10244
            Gradient evaluations: 26
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1810709374064076e-09
     

Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.616864401224104e-10
            Iterations: 65
            Function evaluations: 26455
            Gradient evaluations: 65
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.67490438962725e-05
            Iterations: 240
            Function evaluations: 98160
            Gradient evaluations: 240
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.1297248607554544
            Iterations: 31
            Function evaluations: 12706
            Gradient evaluations: 31
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.089782066321516e-10
            Iterations: 65
            Function evaluations: 26585
            Gradient evaluations: 65
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.83141

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.5564516238351044e-05
            Iterations: 269
            Function evaluations: 114056
            Gradient evaluations: 269
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5005428564569456
            Iterations: 55
            Function evaluations: 23375
            Gradient evaluations: 55
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9707936916520675e-09
            Iterations: 35
            Function evaluations: 14840
            Gradient evaluations: 35
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.4479997069341884e-05
            Iterations: 279
            Function evaluations: 118575
            Gradient evaluations: 279
Optimization terminated successfully    (Exit mode 0)
            Current function value:

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.594888427249916
            Iterations: 52
            Function evaluations: 22307
            Gradient evaluations: 52
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7517876786082072e-09
            Iterations: 38
            Function evaluations: 16264
            Gradient evaluations: 38
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.204497206165216e-05
            Iterations: 299
            Function evaluations: 127973
            Gradient evaluations: 299
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6013617264802384
            Iterations: 47
            Function evaluations: 20162
            Gradient evaluations: 47
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7608304884320207e-09
    

In [55]:
#Exportieren nach CSV
portfolio_returns_df.to_csv("renditensp500_neu_3jahre_f.csv")

## Ergebnis-Erzeugung JSE

In [56]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of               4SI.JO    ABG.JO    ACL.JO    ACT.JO    ADH.JO    AEL.JO  \
Date                                                                     
2005-06-02       NaN  0.001214 -0.006225  0.000000 -0.014598       NaN   
2005-06-03       NaN  0.001333  0.000000  0.000000  0.000000       NaN   
2005-06-06       NaN  0.003510  0.014953  0.000000  0.007407       NaN   
2005-06-07       NaN  0.006634 -0.035437  0.146154 -0.007353       NaN   
2005-06-08       NaN -0.005392 -0.029721  0.000000  0.007407       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-26  0.000000  0.007235 -0.018018  0.000000  0.005975  0.000426   
2025-05-27  0.000000  0.000178 -0.036697  0.000000  0.003439  0.000000   
2025-05-28  0.042857  0.026828 -0.038095  0.087500  0.022741  0.002128   
2025-05-29 -0.041096  0.016127  0.049505 -0.045977 -0.010966  0.002972   
2025-05-30  0.000000 -0.014051  0.094340 -0.036145 -0.009855 -0.001693   

     

In [57]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 36
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 189
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    JSE_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": JSE_gewichte_equally_weighted,
    "Minimum-Variance": JSE_gewichte_min_variance,
    "Maximum-Diversification": JSE_gewichte_max_diversification,
    "Inverse-Volatility": JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": JSE_gewichte_risk_parity,
    "HRP": JSE_gewichte_hrp,
    "HERC": JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    #Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
# Alle Testperioden zusammenführen
portfolio_returns_df_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_JSE = portfolio_returns_df_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.833480544495767e-05
            Iterations: 112
            Function evaluations: 7617
            Gradient evaluations: 112
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5678566840693864
            Iterations: 36
            Function evaluations: 2485
            Gradient evaluations: 36
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0029101255051479e-09
            Iterations: 43
            Function evaluations: 2924
            Gradient evaluations: 43
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.899720135338704e-05
            Iterations: 137
            Function evaluations: 9316
            Gradient evaluations: 137
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5682

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.8090729153036684
            Iterations: 40
            Function evaluations: 2833
            Gradient evaluations: 40
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.803123300005667e-10
            Iterations: 52
            Function evaluations: 3641
            Gradient evaluations: 52
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.279354758902651e-05
            Iterations: 140
            Function evaluations: 9940
            Gradient evaluations: 140
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.9247227969422447
            Iterations: 42
            Function evaluations: 3020
            Gradient evaluations: 42
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0045029614333952e-09
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3547990299774225e-09
            Iterations: 52
            Function evaluations: 3849
            Gradient evaluations: 52
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.333605203624446e-05
            Iterations: 157
            Function evaluations: 11776
            Gradient evaluations: 157
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.9229162372605857
            Iterations: 42
            Function evaluations: 3188
            Gradient evaluations: 42
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.395571152905718e-09
            Iterations: 52
            Function evaluations: 3901
            Gradient evaluations: 52
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.318233

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.9143722518530174e-05
            Iterations: 160
            Function evaluations: 12481
            Gradient evaluations: 160
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.115441896405705
            Iterations: 43
            Function evaluations: 3391
            Gradient evaluations: 43
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.0938671570834433e-09
            Iterations: 63
            Function evaluations: 4914
            Gradient evaluations: 63
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.660419619735268e-05
            Iterations: 162
            Function evaluations: 12474
            Gradient evaluations: 162
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.08

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.5081703612599515
            Iterations: 56
            Function evaluations: 4979
            Gradient evaluations: 56
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.746822550272918e-09
            Iterations: 45
            Function evaluations: 3961
            Gradient evaluations: 45
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5727829408707015e-05
            Iterations: 211
            Function evaluations: 18568
            Gradient evaluations: 211
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.556164710717672
            Iterations: 55
            Function evaluations: 4895
            Gradient evaluations: 55
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.6141763246593415e-09
        

...

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.146060612688794e-05
            Iterations: 196
            Function evaluations: 25088
            Gradient evaluations: 196
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.8364529569620514
            Iterations: 71
            Function evaluations: 9159
            Gradient evaluations: 71
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9585957534890728e-09
            Iterations: 51
            Function evaluations: 6529
            Gradient evaluations: 51
2017-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.231530737423707e-05
            Iterations: 203
            Function evaluations: 26187
            Gradient evaluations: 203
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.84

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.9065861994799413
            Iterations: 65
            Function evaluations: 8646
            Gradient evaluations: 65
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1034558882603458e-09
            Iterations: 58
            Function evaluations: 7657
            Gradient evaluations: 58
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.0937824073366766e-05
            Iterations: 180
            Function evaluations: 23941
            Gradient evaluations: 180
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.9583182972355804
            Iterations: 67
            Function evaluations: 8981
            Gradient evaluations: 67
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.352395236741195e-10
       

Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.958548738794862e-10
            Iterations: 58
            Function evaluations: 8120
            Gradient evaluations: 58
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.128505733179366e-05
            Iterations: 200
            Function evaluations: 28600
            Gradient evaluations: 200
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.111958220144742
            Iterations: 102
            Function evaluations: 14267
            Gradient evaluations: 98
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.062757815533753e-09
            Iterations: 58
            Function evaluations: 8294
            Gradient evaluations: 58
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.177262

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2646689441108966e-05
            Iterations: 199
            Function evaluations: 29652
            Gradient evaluations: 199
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.117230962672459
            Iterations: 88
            Function evaluations: 13231
            Gradient evaluations: 88
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.965538020262236e-09
            Iterations: 44
            Function evaluations: 6556
            Gradient evaluations: 44
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1815953826992243e-05
            Iterations: 195
            Function evaluations: 29055
            Gradient evaluations: 195
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.1

Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.254680937466516
            Iterations: 120
            Function evaluations: 17955
            Gradient evaluations: 116
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.833605718385082e-09
            Iterations: 43
            Function evaluations: 6536
            Gradient evaluations: 43
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9428443173944707e-05
            Iterations: 209
            Function evaluations: 31559
            Gradient evaluations: 209
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.221921161466117
            Iterations: 89
            Function evaluations: 13564
            Gradient evaluations: 89
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.936038047903125e-09
      

In [58]:
#Exportieren nach CSV
portfolio_returns_df_JSE.to_csv("returnsJSE_neu_3jahre_f.csv")

## Ledoit-Wolf Shrinkage

In [59]:
#Definition von Ledoit-Shrinkage

def ledoit_shrink (returns):
    
    shrinkage = LedoitWolf().fit(returns.values)
    
    shrinkage_matrix = pd.DataFrame(
        shrinkage.covariance_,
        index = returns.columns,
        columns = returns.columns)
    
    return shrinkage_matrix

### Auswertung S&P500 mit Ledoit Wolf

In [60]:
returns = df_SP500.pct_change()
returns = returns.where(~df_SP500.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A  AAL  AAP      AAPL      ABBV      ABNB       ABT  \
Date                                                                     
2005-06-02  0.000414  NaN  NaN -0.006452       NaN       NaN -0.001448   
2005-06-03 -0.005374  NaN  NaN -0.044955       NaN       NaN  0.000207   
2005-06-06  0.019119  NaN  NaN -0.008368       NaN       NaN  0.003729   
2005-06-07 -0.012235  NaN  NaN -0.036393       NaN       NaN  0.006398   
2005-06-08  0.009496  NaN  NaN  0.010399       NaN       NaN -0.002051   
...              ...  ...  ...       ...       ...       ...       ...   
2025-05-23 -0.010485  NaN  NaN -0.030244  0.003944 -0.007286 -0.001521   
2025-05-27  0.025154  NaN  NaN  0.025298  0.013424  0.021149  0.012490   
2025-05-28 -0.003415  NaN  NaN  0.001049 -0.014161 -0.005641 -0.006920   
2025-05-29  0.021645  NaN  NaN -0.002345  0.013818 -0.002409  0.006287   
2025-05-30 -0.012006  NaN  NaN  0.004501  0.002640  0.004986  0.005495   

     

In [61]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 36
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 189
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    shrinked_gewichte_equally_weighted = equally_weighted(training_daten)
    shrinked_gewichte_min_variance= min_variance_opt(cov_matrix)
    shrinked_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    shrinked_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    shrinked_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    shrinked_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    shrinked_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": shrinked_gewichte_equally_weighted,
    "Minimum-Variance": shrinked_gewichte_min_variance,
    "Maximum-Diversification": shrinked_gewichte_max_diversification,
    "Inverse-Volatility": shrinked_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": shrinked_gewichte_risk_parity,
    "HRP": shrinked_gewichte_hrp,
    "HERC": shrinked_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_shrinked = pd.concat(portfolio_returns_list)
portfolio_returns_df_shrinked = portfolio_returns_df_shrinked.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.027483279589741e-05
            Iterations: 264
            Function evaluations: 68640
            Gradient evaluations: 264
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6212183083901697
            Iterations: 53
            Function evaluations: 13835
            Gradient evaluations: 53
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.8754509271376374e-09
            Iterations: 30
            Function evaluations: 7801
            Gradient evaluations: 30
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.20992209111112e-05
            Iterations: 261
            Function evaluations: 67599
            Gradient evaluations: 261
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.57

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9807051565378282
            Iterations: 27
            Function evaluations: 7181
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.1238996100870635e-10
            Iterations: 80
            Function evaluations: 21200
            Gradient evaluations: 80
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00010974187591447138
            Iterations: 164
            Function evaluations: 43624
            Gradient evaluations: 164
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.979511320928422
            Iterations: 28
            Function evaluations: 7475
            Gradient evaluations: 28
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.2884751716956043e-10
      

Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.0375734866241787e-10
            Iterations: 78
            Function evaluations: 21450
            Gradient evaluations: 78
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00011756475552027196
            Iterations: 135
            Function evaluations: 37125
            Gradient evaluations: 135
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.9259538820575093
            Iterations: 27
            Function evaluations: 7451
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.010757220055621e-10
            Iterations: 78
            Function evaluations: 21450
            Gradient evaluations: 78
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.000

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0001033629367679537
            Iterations: 156
            Function evaluations: 44616
            Gradient evaluations: 156
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8503403895288753
            Iterations: 29
            Function evaluations: 8321
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.002558061875383e-10
            Iterations: 78
            Function evaluations: 22309
            Gradient evaluations: 78
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00010267524026053373
            Iterations: 123
            Function evaluations: 35424
            Gradient evaluations: 123
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0048440217745926
            Iterations: 29
            Function evaluations: 8989
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.720449995198641e-10
            Iterations: 57
            Function evaluations: 17614
            Gradient evaluations: 57
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.8656753361596173e-05
            Iterations: 207
            Function evaluations: 64377
            Gradient evaluations: 207
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0049702857967873
            Iterations: 29
            Function evaluations: 9045
            Gradient evaluations: 29
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2451922518840573e-09
      

...

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.589039707033979e-05
            Iterations: 191
            Function evaluations: 74490
            Gradient evaluations: 191
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.150576864757984
            Iterations: 32
            Function evaluations: 12511
            Gradient evaluations: 32
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.210438119188038e-10
            Iterations: 50
            Function evaluations: 19500
            Gradient evaluations: 50
2017-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.777885849533981e-05
            Iterations: 200
            Function evaluations: 78201
            Gradient evaluations: 200
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.13

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.226643091733102
            Iterations: 28
            Function evaluations: 10975
            Gradient evaluations: 28
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1294689681479079e-09
            Iterations: 59
            Function evaluations: 23070
            Gradient evaluations: 59
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.199721835736043e-05
            Iterations: 212
            Function evaluations: 83316
            Gradient evaluations: 212
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.2284910857836997
            Iterations: 28
            Function evaluations: 11032
            Gradient evaluations: 28
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1274623667154388e-09
     

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2577676081746452e-09
            Iterations: 55
            Function evaluations: 22386
            Gradient evaluations: 55
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.517296844218336e-05
            Iterations: 206
            Function evaluations: 84254
            Gradient evaluations: 206
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.1563114777918773
            Iterations: 30
            Function evaluations: 12299
            Gradient evaluations: 30
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1990141257273267e-09
            Iterations: 55
            Function evaluations: 22496
            Gradient evaluations: 55
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.68

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.4948916310872906e-05
            Iterations: 266
            Function evaluations: 112785
            Gradient evaluations: 266
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5220625190788604
            Iterations: 62
            Function evaluations: 26350
            Gradient evaluations: 62
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.923440251890458e-09
            Iterations: 35
            Function evaluations: 14840
            Gradient evaluations: 35
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.401265991192629e-05
            Iterations: 261
            Function evaluations: 110926
            Gradient evaluations: 261
Optimization terminated successfully    (Exit mode 0)
            Current function value: -

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.612288588739486
            Iterations: 61
            Function evaluations: 24453
            Gradient evaluations: 57
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.717958549048076e-09
            Iterations: 38
            Function evaluations: 16264
            Gradient evaluations: 38
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.157628220851824e-05
            Iterations: 297
            Function evaluations: 127116
            Gradient evaluations: 297
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.619174159083271
            Iterations: 54
            Function evaluations: 21447
            Gradient evaluations: 50
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.7257819837899684e-09
      

In [62]:
#Exportieren nach CSV
portfolio_returns_df_shrinked.to_csv("returnssp500shrinked_neu_3jahre_f.csv")

### Auswertung JSE mit Ledoit Wolf

In [63]:
returns = df_JSE.pct_change()
returns = returns.where(~df_JSE.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of               4SI.JO    ABG.JO    ACL.JO    ACT.JO    ADH.JO    AEL.JO  \
Date                                                                     
2005-06-02       NaN  0.001214 -0.006225  0.000000 -0.014598       NaN   
2005-06-03       NaN  0.001333  0.000000  0.000000  0.000000       NaN   
2005-06-06       NaN  0.003510  0.014953  0.000000  0.007407       NaN   
2005-06-07       NaN  0.006634 -0.035437  0.146154 -0.007353       NaN   
2005-06-08       NaN -0.005392 -0.029721  0.000000  0.007407       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-26  0.000000  0.007235 -0.018018  0.000000  0.005975  0.000426   
2025-05-27  0.000000  0.000178 -0.036697  0.000000  0.003439  0.000000   
2025-05-28  0.042857  0.026828 -0.038095  0.087500  0.022741  0.002128   
2025-05-29 -0.041096  0.016127  0.049505 -0.045977 -0.010966  0.002972   
2025-05-30  0.000000 -0.014051  0.094340 -0.036145 -0.009855 -0.001693   

     

In [64]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 36
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 189
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    shrinked_JSE_gewichte_equally_weighted = equally_weighted(training_daten)
    shrinked_JSE_gewichte_min_variance= min_variance_opt(cov_matrix)
    shrinked_JSE_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    shrinked_JSE_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    shrinked_JSE_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    shrinked_JSE_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    shrinked_JSE_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": shrinked_JSE_gewichte_equally_weighted,
    "Minimum-Variance": shrinked_JSE_gewichte_min_variance,
    "Maximum-Diversification": shrinked_JSE_gewichte_max_diversification,
    "Inverse-Volatility": shrinked_JSE_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": shrinked_JSE_gewichte_risk_parity,
    "HRP": shrinked_JSE_gewichte_hrp,
    "HERC": shrinked_JSE_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_shrinked_JSE = pd.concat(portfolio_returns_list)
portfolio_returns_df_shrinked_JSE = portfolio_returns_df_shrinked_JSE.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.736052863323831e-05
            Iterations: 141
            Function evaluations: 9588
            Gradient evaluations: 141
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6117178265077983
            Iterations: 38
            Function evaluations: 2621
            Gradient evaluations: 38
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.997567417088884e-10
            Iterations: 43
            Function evaluations: 2924
            Gradient evaluations: 43
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.8033449570930836e-05
            Iterations: 134
            Function evaluations: 9112
            Gradient evaluations: 134
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.6114

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.8481339524154703
            Iterations: 38
            Function evaluations: 2695
            Gradient evaluations: 38
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.658350689295918e-10
            Iterations: 53
            Function evaluations: 3710
            Gradient evaluations: 53
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.166169047415446e-05
            Iterations: 138
            Function evaluations: 9799
            Gradient evaluations: 138
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.9678696327438874
            Iterations: 42
            Function evaluations: 3024
            Gradient evaluations: 42
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.615609013274571e-10
          

Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1850529702413518e-09
            Iterations: 52
            Function evaluations: 3848
            Gradient evaluations: 52
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.244738770045927e-05
            Iterations: 152
            Function evaluations: 11400
            Gradient evaluations: 152
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.9642297378983873
            Iterations: 42
            Function evaluations: 3189
            Gradient evaluations: 42
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.2398633344346603e-09
            Iterations: 52
            Function evaluations: 3900
            Gradient evaluations: 52
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.22174

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.837156960304648e-05
            Iterations: 154
            Function evaluations: 12012
            Gradient evaluations: 154
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.1692372849298596
            Iterations: 42
            Function evaluations: 3315
            Gradient evaluations: 42
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.101978983125467e-10
            Iterations: 64
            Function evaluations: 4992
            Gradient evaluations: 64
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.6039127306178265e-05
            Iterations: 137
            Function evaluations: 10550
            Gradient evaluations: 137
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.13

Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.588747899174835
            Iterations: 58
            Function evaluations: 5160
            Gradient evaluations: 58
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.2513792522108066e-09
            Iterations: 45
            Function evaluations: 3961
            Gradient evaluations: 45
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5792886265557186e-05
            Iterations: 197
            Function evaluations: 17336
            Gradient evaluations: 197
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.6390643468389774
            Iterations: 57
            Function evaluations: 5074
            Gradient evaluations: 57
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.115329403614023e-09
        

...

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.995971750279398e-05
            Iterations: 157
            Function evaluations: 20096
            Gradient evaluations: 157
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.982203641240491
            Iterations: 85
            Function evaluations: 10970
            Gradient evaluations: 85
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.4780287952644683e-09
            Iterations: 52
            Function evaluations: 6656
            Gradient evaluations: 52
2017-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.097342387221166e-05
            Iterations: 175
            Function evaluations: 22576
            Gradient evaluations: 175
Optimization terminated successfully    (Exit mode 0)
            Current function value: -3.99

Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.032502355958461
            Iterations: 74
            Function evaluations: 9845
            Gradient evaluations: 74
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.24527480792031e-10
            Iterations: 58
            Function evaluations: 7657
            Gradient evaluations: 58
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.89855194043502e-05
            Iterations: 167
            Function evaluations: 22211
            Gradient evaluations: 167
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.087235935024922
            Iterations: 76
            Function evaluations: 10190
            Gradient evaluations: 76
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.896368086004356e-10
            

Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.680380892051179e-10
            Iterations: 58
            Function evaluations: 8120
            Gradient evaluations: 58
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.964363414348601e-05
            Iterations: 161
            Function evaluations: 23024
            Gradient evaluations: 161
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4.231300783358366
            Iterations: 85
            Function evaluations: 11679
            Gradient evaluations: 81
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.175430298576423e-10
            Iterations: 58
            Function evaluations: 8294
            Gradient evaluations: 58
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.0043102

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.2150383851973952e-05
            Iterations: 193
            Function evaluations: 28758
            Gradient evaluations: 193
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.221398202777332
            Iterations: 122
            Function evaluations: 17840
            Gradient evaluations: 118
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.306629734524965e-09
            Iterations: 33
            Function evaluations: 4918
            Gradient evaluations: 33
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.1313453737871498e-05
            Iterations: 187
            Function evaluations: 27864
            Gradient evaluations: 187
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5

Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.340798190486456
            Iterations: 97
            Function evaluations: 14883
            Gradient evaluations: 97
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.175382613300002e-09
            Iterations: 44
            Function evaluations: 6688
            Gradient evaluations: 44
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.9182815929243095e-05
            Iterations: 204
            Function evaluations: 30804
            Gradient evaluations: 204
Optimization terminated successfully    (Exit mode 0)
            Current function value: -5.303837853763517
            Iterations: 95
            Function evaluations: 14473
            Gradient evaluations: 95
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.2752094422979676e-09
       

In [65]:
#Exportieren nach CSV
portfolio_returns_df_shrinked_JSE.to_csv("returnsJSEshrinked_neu_3jahre_f.csv")

### Reduzierter S&P500-Bestand

In [66]:
returns = df_SP500_reduziert.pct_change()
returns = returns.where(~df_SP500_reduziert.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A      AAPL       ABT       ACN       ADI       ADM  \
Date                                                                     
2005-06-02  0.000414 -0.006452       NaN       NaN       NaN       NaN   
2005-06-03 -0.005374 -0.044955       NaN       NaN       NaN       NaN   
2005-06-06  0.019119 -0.008368       NaN       NaN       NaN       NaN   
2005-06-07 -0.012235 -0.036393       NaN       NaN       NaN       NaN   
2005-06-08  0.009496  0.010399       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-23 -0.010485 -0.030244 -0.001521 -0.021710 -0.006889 -0.000417   
2025-05-27  0.025154  0.025298  0.012490  0.018897  0.030503  0.014393   
2025-05-28 -0.003415  0.001049 -0.006920  0.001775 -0.005856 -0.005758   
2025-05-29  0.021645 -0.002345  0.006287  0.005507  0.000603  0.000414   
2025-05-30 -0.012006  0.004501  0.005495 -0.002864 -0.008204 -0.002067   

     

In [67]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 36
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 189
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = training_daten.cov()
    corr_matrix = training_daten.corr()
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    reduziert_gewichte_equally_weighted = equally_weighted(training_daten)
    reduziert_gewichte_min_variance= min_variance_opt(cov_matrix)
    reduziert_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    reduziert_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    reduziert_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    reduziert_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    reduziert_gewichte_herc = herc(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": reduziert_gewichte_equally_weighted,
    "Minimum-Variance": reduziert_gewichte_min_variance,
    "Maximum-Diversification": reduziert_gewichte_max_diversification,
    "Inverse-Volatility": reduziert_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": reduziert_gewichte_risk_parity,
    "HRP": reduziert_gewichte_hrp,
    "HERC": reduziert_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_reduziert = pd.concat(portfolio_returns_list)
portfolio_returns_df_reduziert = portfolio_returns_df_reduziert.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.6639251243723945e-05
            Iterations: 199
            Function evaluations: 20497
            Gradient evaluations: 199
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.375725919243715
            Iterations: 32
            Function evaluations: 3329
            Gradient evaluations: 32
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.836021875424328e-10
            Iterations: 42
            Function evaluations: 4327
            Gradient evaluations: 42
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.9200980538340834e-05
            Iterations: 207
            Function evaluations: 21321
            Gradient evaluations: 207
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.34

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8057216505654783
            Iterations: 21
            Function evaluations: 2434
            Gradient evaluations: 21
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.704210459502091e-10
            Iterations: 83
            Function evaluations: 9545
            Gradient evaluations: 83
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00012472648259119256
            Iterations: 127
            Function evaluations: 14605
            Gradient evaluations: 127
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.804734738773179
            Iterations: 22
            Function evaluations: 2549
            Gradient evaluations: 22
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5826168686152484e-10
        

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5044726101337994e-10
            Iterations: 82
            Function evaluations: 10005
            Gradient evaluations: 82
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00013105642764661127
            Iterations: 134
            Function evaluations: 16348
            Gradient evaluations: 134
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.7666972493734863
            Iterations: 24
            Function evaluations: 2945
            Gradient evaluations: 24
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.5427605741860164e-10
            Iterations: 82
            Function evaluations: 10005
            Gradient evaluations: 82
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00011218545382524099
            Iterations: 111
            Function evaluations: 14097
            Gradient evaluations: 111
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6567177750428248
            Iterations: 26
            Function evaluations: 3321
            Gradient evaluations: 26
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.021106319375812e-10
            Iterations: 83
            Function evaluations: 10541
            Gradient evaluations: 83
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0001111073101527307
            Iterations: 116
            Function evaluations: 14732
            Gradient evaluations: 116
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8016572951692993
            Iterations: 22
            Function evaluations: 3166
            Gradient evaluations: 22
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1548402772317653e-09
            Iterations: 52
            Function evaluations: 7436
            Gradient evaluations: 52
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.281706005456162e-05
            Iterations: 180
            Function evaluations: 25741
            Gradient evaluations: 180
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.809098056997133
            Iterations: 23
            Function evaluations: 3308
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1546176891796491e-09
        

...

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.599510454423817e-05
            Iterations: 189
            Function evaluations: 35154
            Gradient evaluations: 189
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0405760454380713
            Iterations: 23
            Function evaluations: 4298
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.895384455218737e-10
            Iterations: 61
            Function evaluations: 11347
            Gradient evaluations: 61
2017-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.808834746662519e-05
            Iterations: 189
            Function evaluations: 35154
            Gradient evaluations: 189
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.02

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0960501333430392
            Iterations: 24
            Function evaluations: 4655
            Gradient evaluations: 24
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5243299722469633e-09
            Iterations: 61
            Function evaluations: 11774
            Gradient evaluations: 61
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.462271158749436e-05
            Iterations: 176
            Function evaluations: 34144
            Gradient evaluations: 176
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0930885430058734
            Iterations: 27
            Function evaluations: 5262
            Gradient evaluations: 27
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6190682305361895e-09
      

Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.188366998283894e-10
            Iterations: 69
            Function evaluations: 13731
            Gradient evaluations: 69
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00010785954609274605
            Iterations: 220
            Function evaluations: 43780
            Gradient evaluations: 220
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.035593246865969
            Iterations: 26
            Function evaluations: 5198
            Gradient evaluations: 26
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.617032149201986e-10
            Iterations: 69
            Function evaluations: 13731
            Gradient evaluations: 69
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00010

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.153341588078155e-05
            Iterations: 278
            Function evaluations: 56713
            Gradient evaluations: 278
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.417985279417961
            Iterations: 37
            Function evaluations: 7585
            Gradient evaluations: 37
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.6053965880573806e-09
            Iterations: 38
            Function evaluations: 7753
            Gradient evaluations: 38
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.0223723411535424e-05
            Iterations: 286
            Function evaluations: 58344
            Gradient evaluations: 286
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.38

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.486295046666397
            Iterations: 40
            Function evaluations: 8239
            Gradient evaluations: 40
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1401854749026824e-09
            Iterations: 38
            Function evaluations: 7791
            Gradient evaluations: 38
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.8045192733886324e-05
            Iterations: 240
            Function evaluations: 49201
            Gradient evaluations: 240
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.492620926540648
            Iterations: 33
            Function evaluations: 6798
            Gradient evaluations: 33
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1551162531812066e-09
        

In [68]:
#Exportieren nach CSV
portfolio_returns_df_reduziert.to_csv("returnsreduziert_neu_3jahre_f.csv")

### Reduziert und Ledoit-Shrinkage

In [69]:
returns = df_SP500_reduziert.pct_change()
returns = returns.where(~df_SP500_reduziert.isna())
returns = returns.iloc[1:]
returns.index = pd.to_datetime(returns.index)
returns.info

<bound method DataFrame.info of                    A      AAPL       ABT       ACN       ADI       ADM  \
Date                                                                     
2005-06-02  0.000414 -0.006452       NaN       NaN       NaN       NaN   
2005-06-03 -0.005374 -0.044955       NaN       NaN       NaN       NaN   
2005-06-06  0.019119 -0.008368       NaN       NaN       NaN       NaN   
2005-06-07 -0.012235 -0.036393       NaN       NaN       NaN       NaN   
2005-06-08  0.009496  0.010399       NaN       NaN       NaN       NaN   
...              ...       ...       ...       ...       ...       ...   
2025-05-23 -0.010485 -0.030244 -0.001521 -0.021710 -0.006889 -0.000417   
2025-05-27  0.025154  0.025298  0.012490  0.018897  0.030503  0.014393   
2025-05-28 -0.003415  0.001049 -0.006920  0.001775 -0.005856 -0.005758   
2025-05-29  0.021645 -0.002345  0.006287  0.005507  0.000603  0.000414   
2025-05-30 -0.012006  0.004501  0.005495 -0.002864 -0.008204 -0.002067   

     

In [70]:
#Definieren von Start und Ende des gesamten Datenbestandes
start_datum = pd.Timestamp('2005-06-01')
end_datum = pd.Timestamp('2025-05-31')

#Liste der Portfolio-Tagesrenditen
portfolio_returns_list = []

#Definieren der Trainings- und Testzeitfenster in Monaten
training_fenster = 36
test_fenster = 1
 
#Setzen der Start-und Endzeitpunkte
training_anfang = start_datum
training_ende = start_datum + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
training_ende = training_ende + MonthEnd(0)
test_anfang = training_ende + pd.DateOffset(days = 1)
test_anfang = test_anfang + MonthBegin(0)
test_ende = training_ende + pd.DateOffset(months = test_fenster)
test_ende = test_ende + MonthEnd(0)

#Rollierende Ergebniserzeugung
while training_ende + pd.DateOffset(months = test_fenster) <= end_datum:
    
    print(training_anfang)
    
    #Definition des Trainingsdatensatzes und des Testdatensatzes
    training_daten = returns.loc[training_anfang:training_ende]
    test_daten = returns.loc[test_anfang:test_ende]
    training_daten = training_daten.dropna(axis=1)
    
    #Um aussagekräftige Kovarianzmatrizen erzeugen zu können, werden nur Assets berücksichtigt, die nicht zu viele 0-Renditen im Datensatz haben
    max_null_tage = 189
    null_zaehler = (training_daten == 0).sum()
    valide_tickers = null_zaehler[null_zaehler <= max_null_tage].index
    training_daten = training_daten[valide_tickers]
    
    #Anpassung der Spalten in den Test-Daten passend zu Trainingsdaten (rausfallende Cash-Out, neue ohne Investment)
    vorhandene_assets = test_daten.columns.intersection(training_daten.columns)
    test_daten = test_daten[vorhandene_assets]
    test_daten = test_daten.fillna(0) #Annahme: ab Ausfall keine Performance, Cash-Out
    
    #Erzeugung Matrizen
    cov_matrix = ledoit_shrink(training_daten)
    
    #aus der geschrumpften Matrix die Korrelationsmatrix bauen
    standardabweichung = np.sqrt(np.diag(cov_matrix))
    corr_matrix = cov_matrix / np.outer(standardabweichung, standardabweichung)
    
    #Ermittlung der Gewichte anhand der Trainingsdaten
    reduziert_shrinked_gewichte_equally_weighted = equally_weighted(training_daten)
    reduziert_shrinked_gewichte_min_variance= min_variance_opt(cov_matrix)
    reduziert_shrinked_gewichte_max_diversification = max_diversification_portfolio(cov_matrix)
    reduziert_shrinked_gewichte_inverse_volatility = inverse_volatility_portfolio(cov_matrix)
    reduziert_shrinked_gewichte_risk_parity = get_risk_parity_weights(cov_matrix)
    reduziert_shrinked_gewichte_hrp = hrp_main(corr_matrix, cov_matrix, training_daten)
    reduziert_shrinked_gewichte_herc = herc_ledoit(training_daten)
    
    gewichte_dict = {
    "Equally-Weighted": reduziert_shrinked_gewichte_equally_weighted,
    "Minimum-Variance": reduziert_shrinked_gewichte_min_variance,
    "Maximum-Diversification": reduziert_shrinked_gewichte_max_diversification,
    "Inverse-Volatility": reduziert_shrinked_gewichte_inverse_volatility,
    "Equal-Risk-Contribution": reduziert_shrinked_gewichte_risk_parity,
    "HRP": reduziert_shrinked_gewichte_hrp,
    "HERC": reduziert_shrinked_gewichte_herc
    }
    
    #Ermitteln der Ergebnisse und Überführen in Ergebnis-Tabellen
    test_portfolio_returns = pd.DataFrame(index=test_daten.index)
    for methode, gewichte in gewichte_dict.items():
        test_portfolio_returns[methode] = test_daten @ gewichte
    
    # Ergebnisse speichern
    portfolio_returns_list.append(test_portfolio_returns)
    
    #Weiter rollieren
    training_anfang = training_anfang + pd.DateOffset(months = test_fenster)
    training_anfang = training_anfang + MonthBegin(0)
    training_ende = training_anfang + pd.DateOffset(months = training_fenster) - pd.DateOffset(days = 1)
    training_ende = training_ende + MonthEnd(0)
    test_anfang = training_ende + pd.DateOffset(days = 1)
    test_anfang = test_anfang + MonthBegin(0)
    test_ende = training_ende + pd.DateOffset(months = test_fenster)
    test_ende = test_ende + MonthEnd(0)
    
#Alle Testperioden zusammenführen
portfolio_returns_df_reduziert_shrinked = pd.concat(portfolio_returns_list)
portfolio_returns_df_reduziert_shrinked = portfolio_returns_df_reduziert_shrinked.reset_index().rename(columns={"index": "Date"})
    
print("Prozedur abgeschlossen")

2005-06-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.629268368830814e-05
            Iterations: 201
            Function evaluations: 20703
            Gradient evaluations: 201
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3992689238590783
            Iterations: 35
            Function evaluations: 3641
            Gradient evaluations: 35
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.329669469460295e-10
            Iterations: 42
            Function evaluations: 4327
            Gradient evaluations: 42
2005-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.880376528732524e-05
            Iterations: 214
            Function evaluations: 22042
            Gradient evaluations: 214
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.366

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8281555596236214
            Iterations: 23
            Function evaluations: 2665
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.895555737256659e-10
            Iterations: 74
            Function evaluations: 8511
            Gradient evaluations: 74
2006-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.000124491439001096
            Iterations: 131
            Function evaluations: 15065
            Gradient evaluations: 131
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8275028015066694
            Iterations: 24
            Function evaluations: 2781
            Gradient evaluations: 24
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.6488665177622794e-10
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4086093417952127e-10
            Iterations: 83
            Function evaluations: 10126
            Gradient evaluations: 83
2007-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00013090029498670508
            Iterations: 117
            Function evaluations: 14274
            Gradient evaluations: 117
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.7869738208315098
            Iterations: 24
            Function evaluations: 2945
            Gradient evaluations: 24
Optimization terminated successfully    (Exit mode 0)
            Current function value: 2.4477206679504233e-10
            Iterations: 83
            Function evaluations: 10126
            Gradient evaluations: 83
2007-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00

2008-07-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00011238263620697632
            Iterations: 119
            Function evaluations: 15114
            Gradient evaluations: 119
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.6847620762790334
            Iterations: 25
            Function evaluations: 3194
            Gradient evaluations: 25
Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.860539283589503e-10
            Iterations: 83
            Function evaluations: 10542
            Gradient evaluations: 83
2008-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.00011116557877127004
            Iterations: 110
            Function evaluations: 13971
            Gradient evaluations: 110
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.

Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8145657994271411
            Iterations: 23
            Function evaluations: 3308
            Gradient evaluations: 23
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1355670425478314e-09
            Iterations: 52
            Function evaluations: 7436
            Gradient evaluations: 52
2009-08-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.2695249038278454e-05
            Iterations: 161
            Function evaluations: 23023
            Gradient evaluations: 161
Optimization terminated successfully    (Exit mode 0)
            Current function value: -1.8220650013945194
            Iterations: 22
            Function evaluations: 3166
            Gradient evaluations: 22
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1351299869556192e-09
      

...

2017-10-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.33348504908531e-05
            Iterations: 182
            Function evaluations: 33853
            Gradient evaluations: 182
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0727459686923835
            Iterations: 24
            Function evaluations: 4484
            Gradient evaluations: 24
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.3913057501982972e-09
            Iterations: 51
            Function evaluations: 9487
            Gradient evaluations: 51
2017-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.53974382179214e-05
            Iterations: 178
            Function evaluations: 33108
            Gradient evaluations: 178
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.0574

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.1245065556869145
            Iterations: 28
            Function evaluations: 5430
            Gradient evaluations: 28
Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.696157151429676e-10
            Iterations: 72
            Function evaluations: 13896
            Gradient evaluations: 72
2018-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.218860603074689e-05
            Iterations: 185
            Function evaluations: 35890
            Gradient evaluations: 185
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.122609646945037
            Iterations: 26
            Function evaluations: 5069
            Gradient evaluations: 26
Optimization terminated successfully    (Exit mode 0)
            Current function value: 8.521473431187045e-10
         

Optimization terminated successfully    (Exit mode 0)
            Current function value: 7.782636072766752e-10
            Iterations: 69
            Function evaluations: 13731
            Gradient evaluations: 69
2019-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0001058353592179952
            Iterations: 183
            Function evaluations: 36418
            Gradient evaluations: 183
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.05908059084439
            Iterations: 24
            Function evaluations: 4800
            Gradient evaluations: 24
Optimization terminated successfully    (Exit mode 0)
            Current function value: 6.301609803234715e-10
            Iterations: 69
            Function evaluations: 13731
            Gradient evaluations: 69
2019-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0001079

2020-11-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5.0825710948754676e-05
            Iterations: 274
            Function evaluations: 55897
            Gradient evaluations: 274
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.436024602464482
            Iterations: 43
            Function evaluations: 8816
            Gradient evaluations: 43
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.5609756710981154e-09
            Iterations: 38
            Function evaluations: 7753
            Gradient evaluations: 38
2020-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.9694875315787085e-05
            Iterations: 274
            Function evaluations: 55896
            Gradient evaluations: 274
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.3

Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5019860463377417
            Iterations: 38
            Function evaluations: 7827
            Gradient evaluations: 38
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1135392574407696e-09
            Iterations: 38
            Function evaluations: 7791
            Gradient evaluations: 38
2021-12-01 00:00:00
Optimization terminated successfully    (Exit mode 0)
            Current function value: 4.7419307268300337e-05
            Iterations: 243
            Function evaluations: 49815
            Gradient evaluations: 243
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2.5091601353448234
            Iterations: 31
            Function evaluations: 6387
            Gradient evaluations: 31
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1.1267676147663199e-09
      

In [71]:
#Exportieren nach CSV
portfolio_returns_df_reduziert_shrinked.to_csv("returnsreduziertshrinked_neu_3jahre_f.csv")